## Population projections statistics

Notebook environment to migrate TIF files to CF compliant CoG

In [22]:
# Import standard packages
import os
import glob
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP5"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname, "data", "LAU_results")

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")


In [3]:
# List all .parquet files
parquet_files = list(ds_dir.glob("*.parquet"))
parquet_files

[WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_Mean_spring_tide.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_RP_1.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_RP_100.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_RP_1000.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_SLR_High_end_2100.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_SLR_High_end_2150.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_SLR_SSP126_2100.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_SLR_SSP245_2050.parquet'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/LAU_results/HIGH_DEFENDED_MAPS_SLR_SS

In [31]:
# Open first parquet file to check
df = gpd.read_parquet(parquet_files[-1])
df

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,geometry,Not Flooded,0-0.5m,0.5-1.0m,1.0-1.5m,1.5-2.0m,2.0-2.5m,2.5m+
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt","POLYGON ((4258927.295 3320011.519, 4259193.583...",17035,0,0,0,0,0,0
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,"POLYGON ((4264556.862 3317435.819, 4265677.354...",1894,0,0,0,0,0,0
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,"POLYGON ((4275479.269 3295636.790, 4275328.930...",5249,0,0,0,0,0,0
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,"POLYGON ((4250468.027 3309420.666, 4250432.180...",1299,0,0,0,0,0,0
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,"POLYGON ((4272425.673 3310568.711, 4274136.202...",1857,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12011,FR_97614,FR,FRY5,FRY50,97614,Ouangani,"POLYGON ((8727685.023 -2785490.190, 8727139.84...",0,0,0,0,0,0,0
12012,FR_97616,FR,FRY5,FRY50,97616,Sada,"POLYGON ((8727114.505 -2787670.745, 8725517.75...",0,0,0,0,0,0,0
12013,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,"POLYGON ((8730964.436 -2773602.954, 8730972.75...",0,0,0,0,0,0,0
12014,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,"MULTIPOLYGON (((8745852.102 -2769772.861, 8742...",0,0,0,0,0,0,0


In [15]:
# Load LAU's from parquet
LAU = gpd.read_parquet(r'p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\LAU_2020_NUTS_2021_01M_3035_CM.parquet')
LAU

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
1192,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,DE_03361001,"POLYGON ((4258927.295 3320011.519, 4259193.583..."
1193,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,DE_03361002,"POLYGON ((4264556.862 3317435.819, 4265677.354..."
1195,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,DE_03361003,"POLYGON ((4275479.269 3295636.790, 4275328.930..."
1196,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,DE_03361004,"POLYGON ((4250468.027 3309420.666, 4250432.180..."
1197,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,DE_03361005,"POLYGON ((4272425.673 3310568.711, 4274136.202..."
...,...,...,...,...,...,...,...,...,...,...,...,...
97667,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,FR_97614,"POLYGON ((8727685.023 -2785490.190, 8727139.84..."
97669,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,FR_97616,"POLYGON ((8727114.505 -2787670.745, 8725517.75..."
97670,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,FR_97617,"POLYGON ((8730964.436 -2773602.954, 8730972.75..."
97660,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,FR_97608,"MULTIPOLYGON (((8745852.102 -2769772.861, 8742..."


In [47]:
duplicates_df = df[df.duplicated(subset='GISCO_ID', keep=False)]
duplicates_df

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,geometry,Not Flooded,0-0.5m,0.5-1.0m,1.0-1.5m,1.5-2.0m,2.0-2.5m,2.5m+
392,BE_11022,BE,BE21,BE211,11022,Kalmthout,"POLYGON ((3941307.611 3160003.860, 3940739.512...",14065,0,0,0,0,0,0
393,BE_11022,BE,BE21,BE211,11022,Kalmthout,"POLYGON ((3941307.611 3160003.860, 3940739.512...",15891,3,4,5,0,0,0
394,BE_11023,BE,BE21,BE211,11023,Kapellen (Antwerpen) / Kapellen (Anvers),"POLYGON ((3938830.333 3152949.208, 3939564.591...",18896,0,0,0,0,0,0
395,BE_11023,BE,BE21,BE211,11023,Kapellen (Antwerpen) / Kapellen (Anvers),"POLYGON ((3938830.333 3152949.208, 3939564.591...",63,0,0,0,0,0,0
396,BE_13004,BE,BE21,BE213,13004,Beerse,"POLYGON ((3961237.089 3143987.559, 3960823.337...",2504,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11887,SE_0184,SE,SE11,SE110,0184,Solna,"POLYGON ((4774310.191 4057937.597, 4774906.280...",3096,0,0,0,0,0,0
11888,SE_0191,SE,SE11,SE110,0191,Sigtuna,"POLYGON ((4762207.487 4077020.980, 4761445.001...",0,0,0,0,0,0,0
11889,SE_0191,SE,SE11,SE110,0191,Sigtuna,"POLYGON ((4762207.487 4077020.980, 4761445.001...",12629,0,0,0,0,0,0
11931,SE_1435,SE,SE23,SE232,1435,Tanum,"MULTIPOLYGON (((4420235.779 3969319.560, 44202...",0,0,0,0,0,0,0


In [52]:
# Get all data from df with LAU_ID == 11022
df[df['LAU_ID'] == '11022']

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,geometry,Not Flooded,0-0.5m,0.5-1.0m,1.0-1.5m,1.5-2.0m,2.0-2.5m,2.5m+
392,BE_11022,BE,BE21,BE211,11022,Kalmthout,"POLYGON ((3941307.611 3160003.860, 3940739.512...",14065,0,0,0,0,0,0
393,BE_11022,BE,BE21,BE211,11022,Kalmthout,"POLYGON ((3941307.611 3160003.860, 3940739.512...",15891,3,4,5,0,0,0
3307,ES_11022,ES,ES61,ES612,11022,"Línea de la Concepción, La","POLYGON ((2935580.368 1595208.559, 2936822.850...",14065,0,0,0,0,0,0
3308,ES_11022,ES,ES61,ES612,11022,"Línea de la Concepción, La","POLYGON ((2935580.368 1595208.559, 2936822.850...",15891,3,4,5,0,0,0


In [49]:
duplicates_LAU = LAU[LAU.duplicated(subset='LAU_ID', keep=False)]
duplicates_LAU

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
13170,BE_11022,BE,BE21,BE211,11022,Kalmthout,18899.0,317.248737,59.571553,2020,BE_11022,"POLYGON ((3941307.611 3160003.860, 3940739.512..."
13181,BE_11023,BE,BE21,BE211,11023,Kapellen (Antwerpen) / Kapellen (Anvers),27525.0,739.310910,37.230615,2020,BE_11023,"POLYGON ((3938830.333 3152949.208, 3939564.591..."
13191,BE_13004,BE,BE21,BE213,13004,Beerse,18141.0,486.611610,37.280245,2020,BE_13004,"POLYGON ((3961237.089 3143987.559, 3960823.337..."
13220,BE_13014,BE,BE21,BE213,13014,Hoogstraten,21440.0,203.299077,105.460390,2020,BE_13014,"POLYGON ((3955577.373 3165950.733, 3956428.146..."
13264,BE_11024,BE,BE21,BE211,11024,Kontich,21204.0,888.205625,23.872850,2020,BE_11024,"POLYGON ((3933690.477 3123774.095, 3932965.823..."
...,...,...,...,...,...,...,...,...,...,...,...,...
92517,SE_0184,SE,SE11,SE110,0184,Solna,82429.0,3930.368893,20.972332,2020,SE_0184,"POLYGON ((4774310.191 4057937.597, 4774906.280..."
92521,SE_0191,SE,SE11,SE110,0191,Sigtuna,48964.0,139.735702,350.404367,2020,SE_0191,"POLYGON ((4762207.487 4077020.980, 4761445.001..."
94524,RO_160911,RO,RO22,RO225,160911,Murighiol,3410.0,4.188105,814.210668,2020,RO_160911,"POLYGON ((5827684.246 2625389.301, 5827719.608..."
92670,RO_62360,RO,RO22,RO223,62360,Oraş Murfatlar,11120.0,167.010355,66.582698,2020,RO_62360,"POLYGON ((5782410.856 2524856.461, 5782657.242..."


In [50]:
duplicates_LAU = LAU[LAU.duplicated(subset='GISCO_ID', keep=False)]
duplicates_LAU

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry


In [54]:
len(df) - len(duplicates_df)


11386

In [55]:
len(LAU)

11965

In [81]:
# Check which unique GISCO_ID's are missing in the unique GISCO_ID's of LAU
LAU_unique = LAU['LAU_ID'].unique()
df_unique = df['LAU_ID'].unique()

missing_GISCO_ID = [GISCO_ID for GISCO_ID in df_unique if GISCO_ID not in LAU_unique]
missing_GISCO_ID


[]

In [23]:
# Drop all unique GISCO_ID's from df, keep duplicates
df_dub = df[df['GISCO_ID'].isin(missing_GISCO_ID)]
df_dub

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,geometry,Not Flooded,0-0.5m,0.5-1.0m,1.0-1.5m,1.5-2.0m,2.0-2.5m,2.5m+


In [65]:
ss_LAU = LAU[20:40]
ss_df = df[20:39]

ss_df = ss_df[['GISCO_ID'] + list(ss_df.columns[-7:])]

# Select only last 7 columns of ss_df
ss_merge = ss_LAU.merge(ss_df, on='GISCO_ID', how='outer')
ss_merge

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry,Not Flooded,0-0.5m,0.5-1.0m,1.0-1.5m,1.5-2.0m,2.0-2.5m,2.5m+
0,DE_03352002,DE,DE93,DE932,03352002,Armstorf,652.0,16.436231,39.668462,2020,DE_03352002,"MULTIPOLYGON (((4263951.769 3387005.922, 42633...",556.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DE_03352004,DE,DE93,DE932,03352004,Belum,812.0,31.785689,25.546088,2020,DE_03352004,"POLYGON ((4257112.338 3413572.939, 4257428.868...",8.0,30.0,57.0,87.0,126.0,190.0,267.0
2,DE_03352008,DE,DE93,DE932,03352008,Bülkau,866.0,37.662378,22.993768,2020,DE_03352008,"POLYGON ((4255023.116 3398212.927, 4252104.910...",48.0,151.0,336.0,314.0,68.0,0.0,0.0
3,DE_03352011,DE,DE93,DE932,03352011,"Cuxhaven, Stadt",48164.0,297.154399,162.084089,2020,DE_03352011,"POLYGON ((4244009.888 3414557.200, 4244322.675...",14386.0,2685.0,3288.0,2571.0,1840.0,1251.0,289.0
4,DE_03352020,DE,DE93,DE932,03352020,Hechthausen,3453.0,112.322875,30.741734,2020,DE_03352020,"POLYGON ((4266481.358 3393275.809, 4269346.299...",3039.0,0.0,0.0,0.0,0.0,0.0,0.0
5,DE_03352022,DE,DE93,DE932,03352022,"Hemmoor, Stadt",8663.0,191.729271,45.183503,2020,DE_03352022,"POLYGON ((4261315.823 3401069.287, 4262064.833...",6817.0,22.0,0.0,0.0,0.0,0.0,0.0
6,DE_03352024,DE,DE93,DE932,03352024,Hollnseth,868.0,41.494974,20.918196,2020,DE_03352024,"POLYGON ((4264619.596 3387528.285, 4264291.983...",1039.0,0.0,0.0,0.0,0.0,0.0,0.0
7,DE_03352025,DE,DE93,DE932,03352025,Ihlienworth,1524.0,37.822151,40.293848,2020,DE_03352025,"POLYGON ((4250251.189 3405595.377, 4252318.243...",45.0,190.0,1214.0,4.0,0.0,0.0,0.0
8,DE_03352029,DE,DE93,DE932,03352029,Lamstedt,3314.0,62.811816,52.760773,2020,DE_03352029,"POLYGON ((4266481.358 3393275.809, 4265725.144...",2290.0,0.0,0.0,0.0,0.0,0.0,0.0
9,DE_03352032,DE,DE93,DE932,03352032,Loxstedt,16311.0,115.253572,141.522729,2020,DE_03352032,"POLYGON ((4220996.531 3367093.547, 4220966.049...",6756.0,252.0,351.0,492.0,643.0,356.0,271.0


In [84]:
# Build a dummy dataframe

LAU_ceed_stats = LAU.copy()

for parquet_file in parquet_files:

    # Retrieve scenario name from file name
    scen_name = Path(parquet_file).stem
    print(f"Processing {scen_name}...")
    
    # Read parquet file
    scen = gpd.read_parquet(parquet_file)

    # Drop all GISCO_ID's duplicates
    scen = scen.drop_duplicates(subset='GISCO_ID', keep='first')

    # Only keep the GISCO_ID and the last 7 columns
    scen = scen[['GISCO_ID'] + list(scen.columns[-7:])]

    # Add scen_name to the column names except GISCO_ID
    scen.columns = ['GISCO_ID'] + [f"{scen_name}\\{col}" for col in scen.columns[1:]]
    
    # Align scenario dataframe with LAU's GISCO_ID using reindex
    scen = scen.set_index('GISCO_ID').reindex(LAU['GISCO_ID'], fill_value=999).reset_index()

    # Merge with LAU_ceed_stats
    LAU_ceed_stats = LAU_ceed_stats.merge(scen, on='GISCO_ID', how='left')

# Fill missing values (flooded stats columns) with 999
LAU_ceed_stats.fillna(999, inplace=True)

LAU_ceed_stats
    

Processing HIGH_DEFENDED_MAPS_Mean_spring_tide...
4960
Processing HIGH_DEFENDED_MAPS_RP_1...
5064
Processing HIGH_DEFENDED_MAPS_RP_100...
5323
Processing HIGH_DEFENDED_MAPS_RP_1000...
11003
Processing HIGH_DEFENDED_MAPS_SLR_High_end_2100...
11722
Processing HIGH_DEFENDED_MAPS_SLR_High_end_2150...
11986
Processing HIGH_DEFENDED_MAPS_SLR_SSP126_2100...
4960
Processing HIGH_DEFENDED_MAPS_SLR_SSP245_2050...
4960
Processing HIGH_DEFENDED_MAPS_SLR_SSP245_2100...
10539
Processing HIGH_DEFENDED_MAPS_SLR_SSP585_2030...
4960
Processing HIGH_DEFENDED_MAPS_SLR_SSP585_2050...
4960
Processing HIGH_DEFENDED_MAPS_SLR_SSP585_2100...
10643
Processing LOW_DEFENDED_MAPS_Mean_spring_tide...
4960
Processing LOW_DEFENDED_MAPS_RP_1...
5064
Processing LOW_DEFENDED_MAPS_RP_100...
10734
Processing LOW_DEFENDED_MAPS_RP_1000...
11722
Processing LOW_DEFENDED_MAPS_SLR_High_end_2100...
11722
Processing LOW_DEFENDED_MAPS_SLR_High_end_2150...
11986
Processing LOW_DEFENDED_MAPS_SLR_SSP126_2100...
9652
Processing LOW_DEF

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,UNDEFENDED_MAPS_SLR_SSP585_2050\1.5-2.0m,UNDEFENDED_MAPS_SLR_SSP585_2050\2.0-2.5m,UNDEFENDED_MAPS_SLR_SSP585_2050\2.5m+,UNDEFENDED_MAPS_SLR_SSP585_2100\Not Flooded,UNDEFENDED_MAPS_SLR_SSP585_2100\0-0.5m,UNDEFENDED_MAPS_SLR_SSP585_2100\0.5-1.0m,UNDEFENDED_MAPS_SLR_SSP585_2100\1.0-1.5m,UNDEFENDED_MAPS_SLR_SSP585_2100\1.5-2.0m,UNDEFENDED_MAPS_SLR_SSP585_2100\2.0-2.5m,UNDEFENDED_MAPS_SLR_SSP585_2100\2.5m+
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,0,0,0,17035,0,0,0,0,0,0
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,0,0,0,1894,0,0,0,0,0,0
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,0,0,0,5249,0,0,0,0,0,0
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,0,0,0,1299,0,0,0,0,0,0
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,0,0,0,1857,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11960,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,0,0,0,0,0,0,0,0,0,0
11961,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,0,0,0,0,0,0,0,0,0,0
11962,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,0,0,0,0,0,0,0,0,0,0
11963,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# Write dummy file to parquet
# LAU_ceed_stats.to_parquet(ds_dir.joinpath("LAU_ceed_stats.parquet"))

In [18]:
# set heatmap dir
heatmap_dir = coclico_data_dir.joinpath(ds_dirname, "data", "flood_heatmaps_buildings")

# List all .tif files
tif_files = list(heatmap_dir.glob("*.tif"))

# Open first tif file to check
ds = rio.open_rasterio(tif_files[-1])
ds

<xarray.DataArray (band: 1, y: 5095, x: 7844)> Size: 160MB
[39965180 values with dtype=float32]
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 63kB -31.27 -31.26 -31.25 ... 34.07 34.08 34.09
  * y            (y) float64 41kB 70.09 70.08 70.07 70.06 ... 27.65 27.65 27.64
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [21]:
# Check if ds is a COG
ds.is_cog

AttributeError: 'DataArray' object has no attribute 'is_cog'

In [20]:
# Write ds to COG
ds.rio.to_raster(ds_dir.joinpath("flood_heatmap_MvS.tif"), compress="DEFLATE", driver='COG')

In [23]:
# Read Ceed parquet file
ceed = gpd.read_parquet(r"p:\11207608-coclico\FULLTRACK_DATA\WP5\data\nuts2_ceed\NL32_CEED.parquet")
ceed    

In [24]:
ceed

id   height     age  type id_source  \
buildings 0      v0.1-NLD.9.29_1-30125  3.99202  1921.0   NaN   3145062   
          1      v0.1-NLD.9.29_1-41348  2.57936  1995.0   NaN   5139506   
          2      v0.1-NLD.9.29_1-30386  3.99912  1921.0   NaN   5131659   
          3      v0.1-NLD.9.29_1-50858  7.15120  1921.0   NaN  15737009   
          4       v0.1-NLD.9.29_1-6337  3.39338  1921.0   NaN  17897485   
...                                ...      ...     ...   ...       ...   
road      65624                   None      NaN     NaN   NaN      None   
air       0                       None      NaN     NaN   NaN      None   
          1                       None      NaN     NaN   NaN      None   
          2                       None      NaN     NaN   NaN      None   
          3                       None      NaN     NaN   NaN      None   

                 type_source  \
buildings 0              NaN   
          1              NaN   
          2              NaN   
          3              NaN   
          4              NaN   
...                      ...   
road      65624          NaN   
air       0              NaN   
          1              NaN   
          2              NaN   
          3              NaN   

                                                          geometry  \
buildings 0      POLYGON ((3993470.786 3244170.146, 3993470.786...   
          1      POLYGON ((3993504.494 3244169.835, 3993504.493...   
          2      POLYGON ((3993442.903 3244164.658, 3993442.902...   
          3      POLYGON ((3993475.053 3244188.084, 3993475.865...   
          4      POLYGON ((3993475.749 3244196.658, 3993475.749...   
...                                                            ...   
road      65624  LINESTRING (3977212.385 3353599.891, 3977205.8...   
air       0      MULTIPOLYGON (((3989847.243 3242433.592, 39898...   
          1      MULTIPOLYGON (((3962542.506 3255596.115, 39625...   
          2      MULTIPOLYGON (((3969894.862 3324744.177, 39697...   
          3      MULTIPOLYGON (((3975183.136 3345760.210, 39749...   

                 building_height  use_type_val             use_type  ...  \
buildings 0             2.809229          23.0  Broad-leaved forest  ...   
          1             2.809229          23.0  Broad-leaved forest  ...   
          2             3.288609          23.0  Broad-leaved forest  ...   
          3             2.809229          23.0  Broad-leaved forest  ...   
          4             2.809229          23.0  Broad-leaved forest  ...   
...                          ...           ...                  ...  ...   
road      65624              NaN           NaN                 None  ...   
air       0                  NaN           NaN                 None  ...   
          1                  NaN           NaN                 None  ...   
          2                  NaN           NaN                 None  ...   
          3                  NaN           NaN                 None  ...   

                generator_output generator_source railway gauge electrified  \
buildings 0                 None             None    None  None        None   
          1                 None             None    None  None        None   
          2                 None             None    None  None        None   
          3                 None             None    None  None        None   
          4                 None             None    None  None        None   
...                          ...              ...     ...   ...         ...   
road      65624             None             None    None  None        None   
air       0                 None             None    None  None        None   
          1                 None             None    None  None        None   
          2                 None             None    None  None        None   
          3                 None             None    None  None        None   

                      highway maxspeed lanes  s

In [26]:
gpkg_dir = ds_dir.parent.joinpath("nuts2_ceed_gpkg")
# Write Ceed to geopackage
ceed.to_file(gpkg_dir.joinpath("NL32_CEED.gpkg"), driver='GPKG')